In [1]:
#! pip3 install pymysql
#! pip3 install paramiko
#! pip3 install pandas
#! pip3 install numpy

In [2]:
import datetime
import random
import string

targetServer = ''
targetPort = 3306
targetdb = ''
targetUser = ''
targetPass = ''
today = datetime.datetime.now().strftime('%Y-%m-%d')

In [ ]:

import pymysql
import pandas as pd

conn = pymysql.connect(host=targetServer, user=targetUser, passwd=targetPass, port=targetPort, db=targetdb)
     #delete all notes and members above id 10
with conn.cursor() as cursor:
     cursor.execute("delete from notes;")
     cursor.execute("delete from members where id > 1;")
     cursor.execute("delete from activities_authorization_approvals;")
     cursor.execute("delete from activities_authorizations;")
conn.commit()
cur = conn.cursor(pymysql.cursors.DictCursor)
cur.execute("select id,name from branches;")
targetBranches = {}
for row in cur:
    #grab the last word of the name and make it lowercase
    name = row['name'].split()[-1].lower()
    targetBranches[name] = row['id']

cur.execute("select * from activities_activities;")
targetAuths = {}
for row in cur:
     name = row['name'].lower()
     targetAuths[name] = row['id']
conn.close()
print(targetBranches)
print(targetAuths)


In [ ]:
#open authTypes.json and read it into a variable in python
import json
members = []
authTypes = []
with open('authTypes.json') as f:
    authTypes = json.load(f)
#open members.json and read it into a variable in python
with open('members.json') as f:
    members = json.load(f)

for auth in authTypes:
    #find the authtype in the targetAuths dictionary by name and set the new_id
    auth['new_id'] = targetAuths[auth['name'].lower()]

authTypesLookup = {}
for auth in authTypes:
    authTypesLookup[auth["id"]] = auth['new_id']

print (authTypes)
print (authTypesLookup)

In [ ]:
# Insert each member into MySQL and store the ID of the new member on the member object
skipNames = {

}
childrenSharedEmail = {

}
skipSecond = {

}
try:
    conn = pymysql.connect(host=targetServer, user=targetUser, passwd=targetPass, port=targetPort, db=targetdb)
    for member in members:
        if(member['email_address'] == ''):
            continue
        if(member['email_address'] in skipNames):
            continue
        cur = conn.cursor(pymysql.cursors.DictCursor)
        cur.execute("select id,sca_name from members where email_address = %s;", member['email_address'])
        memberExists = False
        skip = False
        for row in cur:
            if(member['email_address'] in childrenSharedEmail):
                member['email_address'] = 'p-' + member['email_address']
                break
            if(member['email_address'] in skipSecond):
                skip = True
                break
            memberExists = True
            print(member['sca_name'] + ', ' + member['email_address'] + ', ' + row['sca_name'] + ', ' + str(row['id']))
            member['new_id'] = row['id']
            break
        if(memberExists or skip):
            continue
        with conn.cursor() as cursor:
            # Insert member into the database
            sql = """
            INSERT INTO members (
                `password`, 
                `sca_name`, 
                `first_name`, 
                `middle_name`, 
                `last_name`, 
                `street_address`, 
                `city`, 
                `state`, 
                `zip`, 
                `phone_number`, 
                `email_address`, 
                `membership_number`, 
                `membership_expires_on`, 
                `branch_id`, 
                `background_check_expires_on`, 
                `status`, 
                `verified_date`, 
                `verified_by`, 
                `parent_id`, 
                `mobile_card_token`, 
                `password_token`, 
                `password_token_expires_on`, 
                `last_login`, 
                `last_failed_login`, 
                `failed_login_attempts`, 
                `birth_month`, 
                `birth_year`, 
                `membership_card_path`, 
                `modified`, 
                `created`, 
                `created_by`, 
                `modified_by`, 
                `deleted`)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, NULL, NULL, NULL, NULL, NULL, NULL, 0, %s, %s, NULL, %s, %s, 1, 1, NULL)
            """
            memberStatus = 'active'
            if(member['hidden']):
                memberStatus = 'deactivated'
            else:
                if(member['membership_number']is not None and member['membership_number'] > 0):
                    memberStatus = 'verified'
                
            
            #if the members birth month and year are less than 18 years ago status needs to be youth
            if(member['birth_year'] is not None and member['birth_month'] is not None and member['birth_year'] > 0 and member['birth_month'] > 0):
                memberBirthDate = datetime.datetime(member['birth_year'], member['birth_month'], 1)
                if(memberBirthDate > datetime.datetime.now() - datetime.timedelta(days=18*365)):
                    if(memberStatus == 'verified'):
                        memberStatus = '< 18 member verified'
                    else:
                        memberStatus = 'unverified minor'
            else:
                member['birth_year'] = 2005
                member['birth_month'] = 1
            #lookup the branch id by the last word of the member branch_name lower case in the targetBranches dictionary
            branchId = None
            try:
                branchId = targetBranches[member['branch_name'].split()[-1].lower()]
            except:
                #print('Branch not found - ' + member['branch_name'].split()[-1].lower())
                branchId = branchId = targetBranches['land']
            #print (branchId)
            if(member['membership_number'] is None or member['membership_number'] < 1 or member['membership_number'] == ''):
                member['membership_number'] = None
                member['membership_expires_on'] = None
                member['status'] = 'active'
            else:
                if(member['membership_expires_on'] is not None):
                    member["membership_expires_on"] = datetime.datetime.strptime(member["membership_expires_on"], "%Y-%m-%dT%H:%M:%S%z")
                    member["membership_expires_on"] = member["membership_expires_on"].strftime("%Y-%m-%d")
            if(member['background_check_expires_on'] is not None):
                member["background_check_expires_on"] = datetime.datetime.strptime(member["background_check_expires_on"], "%Y-%m-%dT%H:%M:%S%z")
                member["background_check_expires_on"] = member["background_check_expires_on"].strftime("%Y-%m-%d")
            #execute the insert statement
            try:
                cursor.execute(sql, (
                    member['password'],
                    member['sca_name'],
                    member['first_name'], 
                    member['middle_name'], 
                    member['last_name'], 
                    member['street_address'], 
                    member['city'], 
                    member['state'], 
                    member['zip'], 
                    member['phone_number'], 
                    member['email_address'],
                    member['membership_number'], 
                    member['membership_expires_on'], 
                    branchId, 
                    member['background_check_expires_on'],
                    memberStatus,
                    today.format('Y-m-d'),
                    "1",
                    member['birth_month'], 
                    member['birth_year'],
                    today.format('Y-m-d'),
                    today.format('Y-m-d')
                ))
                # Get the ID of the new member
                member_id = cursor.lastrowid
                # Store the ID on the member object
                member['new_id'] = member_id
            # Commit the transaction
                conn.commit()
            except Exception as e:
                print('Member Failed: ' + member['sca_name'] + ' ' + str(member['id']) + ' - ' + str(e))
                conn.rollback()
    conn.close()
finally:
    print('done')
    #wiesenfeuer
    #weisenfeuer

In [ ]:
# Insert parent name as a note in the notes table if it is not null

try:
    conn = pymysql.connect(host=targetServer, user=targetUser, passwd=targetPass, port=targetPort, db=targetdb)
    for member in members:
        # check if member has a new_id
        if member['notes'] and 'new_id' in member:
            with conn.cursor() as cursor:
                # Insert note into the notes table
                sql_note = """
                INSERT INTO notes (
                    `author_id`,  
                    `created`, 
                    `topic_model`, 
                    `topic_id`, 
                    `subject`, 
                    `body`,
                    `private`)
                VALUES (%s, %s, 'Members', %s, %s, %s, %s)
                """
                note = f"{member['notes']}"
                #print (note)
                cursor.execute(sql_note, (
                    member['new_id'],
                    today.format('Y-m-d'),
                    member['new_id'],
                    'Imported Notes',
                    note,
                    True
                ))
                # Commit the transaction
                conn.commit()
    conn.close()
finally:
    print('done')

In [ ]:
# Insert parent name as a note in the notes table if it is not null

try:
    conn = pymysql.connect(host=targetServer, user=targetUser, passwd=targetPass, port=targetPort, db=targetdb)
    for member in members:
        # check if member has a new_id
        if member['parent_name'] and 'new_id' in member:
            with conn.cursor() as cursor:
                # Insert note into the notes table
                sql_note = """
                INSERT INTO notes (
                    `author_id`,  
                    `created`, 
                    `topic_model`, 
                    `topic_id`, 
                    `subject`, 
                    `body`,
                    `private`)
                VALUES (%s, %s, 'Members', %s, %s, %s, %s)
                """
                note = f"Parent Name: {member['parent_name']}"
                print (note)
                cursor.execute(sql_note, (
                    member['new_id'],
                    today.format('Y-m-d'),
                    member['new_id'],
                    'Import Parent Name',
                    note,
                    True
                ))
                # Commit the transaction
                conn.commit()
    conn.close()
finally:
    print('done')

In [ ]:


#map of members who changed names between authorizations
memberMap = {

}
missingNames = {}
try:
    conn = pymysql.connect(host=targetServer, user=targetUser, passwd=targetPass, port=targetPort, db=targetdb)
    for member in members:
        #if there are authorizations we need to load them up.
        if member['participant_authorization_types'] and 'new_id' in member:
            for auth in member['participant_authorization_types']:

                if auth['authorization_type_id'] in authTypesLookup:
                    authType = authTypesLookup[auth['authorization_type_id']]
                else:
                    print ("auth not found: " + str(auth['authorization_type_id']))
                    continue

                authoByName = auth["authorized_by"]
                authById = 1
                cur = conn.cursor(pymysql.cursors.DictCursor)
                if authoByName in memberMap:
                    authoByName = memberMap[authoByName]
                cur.execute("select id from members where sca_name = %s;", authoByName)
                row = cur.fetchone()
                authById = 1
                if(row is None):
                    missingNames[authoByName] = "yes"
                else:
                    authById = row['id']
                with conn.cursor() as cursor:
                    # Insert authorization into the database
                    sql = """
                    INSERT INTO activities_authorizations (
                        `member_id`, 
                        `activity_id`, 
                        `granted_member_role_id`, 
                        `expires_on`, 
                        `start_on`, 
                        `created`, 
                        `approval_count`, 
                        `status`, 
                        `revoked_reason`, 
                        `revoker_id`, 
                        `is_renewal`)
                    VALUES (%s, %s, NULL, %s, %s, %s, 1, 'Approved', '', NULL, 0)
                    """
                    if auth["start_on"] is None:
                        end_on = datetime.datetime.strptime(auth["expires_on"], "%Y-%m-%dT%H:%M:%S%z")
                        start_on = end_on.replace(year=end_on.year - 4)
                        auth["start_on"] = start_on.strftime("%Y-%m-%d")
                    else:
                        auth["start_on"] = datetime.datetime.strptime(auth["start_on"], "%Y-%m-%dT%H:%M:%S%z").strftime("%Y-%m-%d")
                    if auth["expires_on"] is not None:
                        auth["expires_on"] = datetime.datetime.strptime(auth["expires_on"], "%Y-%m-%dT%H:%M:%S%z").strftime("%Y-%m-%d")
                    cursor.execute(sql,[member["new_id"],authType, auth["expires_on"],auth["start_on"],today])
                    #get Id of the new authorization
                    auth_id = cursor.lastrowid

                    #save the approval 
                    def generate_token(length=32):
                        characters = string.ascii_letters + string.digits
                        return ''.join(random.choice(characters) for _ in range(length))

                    token = generate_token()
                    sql = """
                    INSERT INTO activities_authorization_approvals (
                        `authorization_id`, 
                        `approver_id`, 
                        `authorization_token`, 
                        `requested_on`,
                        `responded_on`,
                        `approved`,
                        `approver_notes`)
                        VALUES (%s, %s, %s, %s, %s, 1, 'Imported')
                    """
                    cursor.execute(sql,[auth_id,authById,token, auth["start_on"],auth["start_on"]])
                conn.commit()
    conn.close()
finally:
    print('done')

In [ ]:


#map of members who changed names between authorizations

rolesMap = {
   4:{'role':1001,'auth':52}, #armoroed
   5:{'role':1002,'auth':53}, #rapier
   6:{'role':1003,'auth':55}, #C&T
   10:{'role':1004,'auth':54}, #Target archery
   30:{'role':1005,'auth':56}, #equestrian
   14:{'role':1006,'auth':58}, #youth rapier
   12:{'role':1007,'auth':57}, #youth chiv & Boffer
   13:{'role':1007,'auth':57}, #youth chiv & Boffer
   8:{'role':1008,'auth':59}, #seige
   7:{'role':1009,'auth':60}, #rapier spear
   11:{'role':1010,'auth':61}, #thrown
   9:{'role':1011,'auth':62}, #combat archery
   27:{'role':1012,'auth':63}, #two handed C&T
   31:{'role':1013,'auth':64}, #wooden lance
   48:{'role':1014,'auth':65}, #reduced armor exp
   22:{'role':1015,'auth':66}, #reduced armor exp
}
try:
    conn = pymysql.connect(host=targetServer, user=targetUser, passwd=targetPass, port=targetPort, db=targetdb)
    for member in members:
        #if there are authorizations we need to load them up.
        if member['roles'] and 'new_id' in member:
            for role in member['roles']:
                if role['_joinData']['role_id'] in rolesMap:
                    role = rolesMap[role['_joinData']['role_id']]
                else:
                    print ("Role Not Found: " + role['name'])
                    continue
                start_on = datetime.datetime.strptime(today, '%Y-%m-%d')
                expires_on = start_on.replace(year=end_on.year + 4)
                expires_on = expires_on.strftime('%Y-%m-%d')
                authById = 1
                with conn.cursor() as cursor:
                    sql = """
                    INSERT INTO member_roles (
                    `member_id`,
                    `role_id`,
                    `start_on`,
                    `expires_on`,
                    `granting_model`,
                    `approver_id`,
                    `modified`,
                    `created`,
                    `created_by`,
                    `modified_by`)
                    VALUES (%s, %s, %s, %s, 'Activities.Authorizations',1, %s, %s, 1, 1)
                    """
                    cursor.execute(sql,[member["new_id"],role['role'], start_on, expires_on, start_on, start_on])
                    #get Id of the new authorization
                    member_role = cursor.lastrowid
                    authType = role['auth']
                    # Insert authorization into the database
                    sql = """
                    INSERT INTO activities_authorizations (
                        `member_id`, 
                        `activity_id`, 
                        `granted_member_role_id`, 
                        `expires_on`, 
                        `start_on`, 
                        `created`, 
                        `approval_count`, 
                        `status`, 
                        `revoked_reason`, 
                        `revoker_id`, 
                        `is_renewal`)
                    VALUES (%s, %s, %s, %s, %s, %s, 1, 'Approved', '', NULL, 0)
                    """
                    cursor.execute(sql,[member["new_id"],authType, member_role ,expires_on, start_on, start_on])
                    #get Id of the new authorization
                    auth_id = cursor.lastrowid

                    #save the approval 
                    def generate_token(length=32):
                        characters = string.ascii_letters + string.digits
                        return ''.join(random.choice(characters) for _ in range(length))

                    token = generate_token()
                    sql = """
                    INSERT INTO activities_authorization_approvals (
                        `authorization_id`, 
                        `approver_id`, 
                        `authorization_token`, 
                        `requested_on`,
                        `responded_on`,
                        `approved`,
                        `approver_notes`)
                        VALUES (%s, %s, %s, %s, %s, 1, 'Imported')
                    """
                    cursor.execute(sql,[auth_id,authById,token, start_on,start_on])
                    sql = """
                        UPDATE member_roles set `granting_id` = %s WHERE id = %s 
                    """
                    cursor.execute(sql,[auth_id,member_role])
                conn.commit()
    conn.close()
finally:
    print('done')